# Pre-processing of Lagain Mars craters for DaCHS DB ingestion

To ingest data from Lagain & Chiara data release -- https://github.com/alagain/martian_crater_database -- into DaCHS, which is a Postgres db, we must first workout the GeoJSON files provided. We should arrive in a flat table.

The workflow is basically:
* read JSON
 * unravel data in one or more tables
* check data
* write CSV

In [1]:
import json
import pandas
print('Pandas version: ', pandas.__version__)

Pandas version:  0.24.1


<div class="alert alert-info">
Where are we?

<pre>
<code>
$ git clone https://github.com/alagain/martian_crater_database.git
$ cd martian_crater_database/Global
$ unzip lagain_db.json.zip
</code>
</pre>
</div>

In [2]:
%ls

lagain_db_filtered.json.zip  Notebook_PreProcLagain.html
lagain_db_filtered.zip       Notebook_PreProcLagain.ipynb
lagain_db.json               notebook.tex
lagain_db.json.zip           README.md
lagain_db.zip


In [3]:
with open('lagain_db.json','r') as fp:
    js = json.load(fp)
    
features = js['features']
print("Number of features: ", len(features))

Number of features:  384582


In [4]:
features[0]

{'type': 'Feature',
 'properties': {'CRATER_ID': '200-007',
  'RADIUS': 500.0,
  'X': 23.671499,
  'Y': -43.584301,
  'TYPE': 1.0,
  'STATUS': 'Valid',
  'LRD_MORPH': None,
  'ORIGIN': None,
  'ADDING': 1.0},
 'geometry': {'type': 'Point', 'coordinates': [23.671499, -43.584301]}}

## Let's check if everything is "Point"
...sanity check actually, because asaik they are all points...

In [5]:
set([f['geometry']['type'] for f in features])

{'Point'}

...all entries are points. OK. Which, together with our "first feature" sample above, I assume that (`X`,`Y`) and `coordinates` have the same values. That being the case, I can simply drop `geometry` in what follows.

## Let's check what is going on in `properties`

In [6]:
df = pandas.read_json(json.dumps([f['properties'] for f in features]))

In [7]:
df.describe(include='all')

,ADDING,CRATER_ID,LRD_MORPH,ORIGIN,RADIUS,STATUS,TYPE,X,Y
count,384582.000000,384582,8480,39204,384582.000000,384582,384582.000000,384582.000000,384582.000000
unique,NaN,384561,4,106,NaN,5,NaN,NaN,NaN
top,NaN,13-009863,SLE,05-000000,NaN,Valid,NaN,NaN,NaN
freq,NaN,2,6680,5857,NaN,288117,NaN,NaN,NaN
mean,0.000793,NaN,NaN,NaN,1778.392579,NaN,1.665770,10.135017,-7.180770
std,0.028150,NaN,NaN,NaN,4295.514407,NaN,1.211242,96.634768,33.612876
min,0.000000,NaN,NaN,NaN,500.000000,NaN,1.000000,-179.996994,-86.699997
25%,0.000000,NaN,NaN,NaN,590.000000,NaN,1.000000,-58.806000,-30.928750
50%,0.000000,NaN,NaN,NaN,765.000000,NaN,1.000000,12.759500,-10.056000
75%,0.000000,NaN,NaN,NaN,1280.000000,NaN,2.000000,89.264749,17.259001


In [8]:
pandas.set_option('display.max_rows',100)
df.sample(100)

,ADDING,CRATER_ID,LRD_MORPH,ORIGIN,RADIUS,STATUS,TYPE,X,Y
168617,0,13-006590,None,None,820,Valid,1,67.170998,4.387000
368718,0,11-010804,None,None,650,Secondary,4,-13.202000,11.433000
325646,0,19-011580,None,19-000132,695,Secondary,4,-26.778000,-12.987001
3487,0,19-020071,None,None,505,Valid,1,-12.381000,-1.276000
160570,0,25-008061,None,None,790,Valid,1,-84.939003,-32.195000
209454,0,21-005681,None,None,1080,Valid,1,61.452999,-27.584000
40933,0,18-012217,None,None,545,Valid,1,-56.749001,-17.180000
127716,0,16-009946,None,None,690,Valid,1,-172.233994,-26.709000
39920,0,11-014789,None,None,540,Valid,1,-21.974001,8.373000
48857,0,14-011179,None,None,550,Valid,1,120.080002,3.865000


# Write down table

In [9]:
df.to_csv('lagain_db.csv', index=False)

In [12]:
!head -n10 lagain_db.csv

ADDING,CRATER_ID,LRD_MORPH,ORIGIN,RADIUS,STATUS,TYPE,X,Y
1,200-007,,,500,Valid,1,23.671499,-43.584300999999996
1,200-008,,,500,Valid,1,68.432404,-39.9505
1,200-009,,,500,Valid,1,177.154007,-0.758709
1,200-010,,,500,Valid,1,34.128899,-31.406401
1,100-001,,,500,Valid,1,-52.9631,47.302601
1,100-002,,,500,Valid,1,-65.612297,33.682999
1,100-003,,,500,Valid,1,22.795299,13.8615
1,100-004,,,500,Valid,1,-60.845299,46.900902
1,200-011,,,505,Valid,1,178.046005,-6.55385
